In [33]:
import urllib.parse
import urllib.request
import pandas as pd
import pickle
import re
import numpy as np
import copy

In [105]:
def NotBruteAtAll(temp):
    res = []
    for t in temp:
        result = []
        t = t.lower()
        words = t.split(" ")
        
        resultStrict = StrictCheck(words) #возвращает матрицу с вероятностями после строгой проверки
        
        tempMatrix = СheckStrictMatrix(resultStrict)
        order, check = GetOrder(tempMatrix), СheckMatrix(tempMatrix)
        #order, check - Это порядок слов и bool значение успешна ли была строгая проверка
        if check:
            for i in range(3):
                result.append(words[order[i]])
        else:
            #Матрицы со словами и соответссвующими частотами для нечетного поиска
            replaceWords = [["","",""],
                            ["","",""],
                            ["","",""]]
            #replaceWords = [words]*3
                        
            replaceValues = [[0,0,0],
                             [0,0,0],
                             [0,0,0]]

            #Проверяем можем ли мы с уверенностью что-то определить
            #Для этого если вероятность значения одного значительно больше остальных, выбираем его.
            #Можно было просто сделать для единственного в столбце, но это на случай если у нас будет одно и то же слово в разных базах
            #Если не нужно, то потом всегда сможем этот блок упростить
            
            #можно сделать сравнивать не с остальными вероятностями, а установить порог, после которого мы считаем значение маловероятным
            order = [None, None, None]
            for j in range(3):
                temp = [resultStrict[0][j], resultStrict[1][j], resultStrict[2][j]]
                indexMax = temp.index(max(temp))
                f = True
                for i in range(3):
                    if i != indexMax and resultStrict[i][j]*10**4 >= resultStrict[indexMax][j]:
                        f = False
                if f: 
                    order[indexMax] = j 
                    replaceWords[indexMax][j] = words[indexMax]
                    replaceValues[indexMax][j] = 1 
            
            #Нужно придумать как сравнивать вероятности строгой проверки и нестрогой
            #Можно просто сразу все нестрогие делить на 1000 и если они все еще будут больше - тогда все ок
            #так, просто предположение
            
            #Оставшиеся слова, не определенные в строгой проверке, проверяем через расстояние между словами
            for i in range(3):
                if order[i] is None:
                    for j in range(3):
                        if j not in order:
                            replaceWords[i][j], replaceValues[i][j] = forReplaceCheck(words[i], bases[j])

            #Проверка по грамаматике, возвращает матрицу с вероятностями что это слово та или иная часть ФИО
            resultGramma = GrammaCheck(words) 
            
            #Складываем все полученное в одну матрицу
            resForNow = [[0,0,0],
                        [0,0,0],
                        [0,0,0]]
            for i in range(3):
                for j in range(3):
                    resForNow[i][j] = replaceValues[i][j] + resultGramma[i][j]

            #Пока выбираем по максимальному значению с обработкой неоднозначных ситуаций
            
            tempMatrix = СheckStrictMatrix(resForNow)
            if СheckMatrix(tempMatrix):
                order = GetOrder(tempMatrix)
            else:
                order = ComplexOrder(tempMatrix)
            
            for i in range(3):
                tmp = replaceWords[order.index(i)][i]
                if tmp == "": tmp = words[order[i]]
                result.append(tmp)

        SetStatistics(order)
        final = ""
        for r in result:
            final += r.title() + " " 
        final = final[:-1]
        res.append(final)
        #result = result[0].title() + " " + result[1].title() + " " + result[2].title()
        #res.append(result)
    return res

In [106]:
def ComplexOrder(matrix):
    
    #Возможно именно сдесь реализовать поиск не по абсолютно максимальным, а по значительно максимальным, 
    # т.е. если значение первышает остальные в 10**n степени
    
    #Ищем индексы максимальных элементов по строкам и столбцам
    order = [None, None, None] 
    maxRows = [0,0,0]
    maxColumns = [0,0,0]
    for i in range(3):
        for j in range(3):
            if matrix[i][j]>matrix[i][maxRows[i]]: 
                maxRows[i] = j
            if matrix[j][i]>matrix[maxColumns[i]][i]: 
                maxColumns[i] = j
    #print(maxRows)
    #print(maxColumns)

    #Если один элемент максимален и в своей строке и в своем столбце, то считаем это значение правильным, "забиваем" его
    for i in range(3):
        if maxRows.count(maxRows[i])==1 and maxColumns.count(i)==1:
            order[i] = maxRows[i]
    n = order.count(None)
    if n==0:
        return order
    elif n==1:
        for i in range(3):
            if i not in order:
                order[order.index(None)] = i
    elif n == 2:
        #Индекс "забитого" элемента
        indexRow = 0
        indexColumn = 0
        for i in range(3): 
            if (order[i] is not None):  
                indexRow = i
                indexColumn = order[i]
        #Сравнивать по полусумме
        #зануляем ненужную строку и столбец из матрицы

        set1 = [[None,None],[None,None]] #Координаты двух точек первого варинта
        set2 = [[None,None],[None,None]] #Координаты двух точек второго варианта
        for i in range(3):
             if i!=indexRow:
                for j in range(3):
                    if j!=indexColumn:
                        if set1[0][0] is None and set1[0][1] is None: 
                            set1[0][0] = i
                            set1[0][1] = j
                            set2[0][0] = i
                            set2[1][1] = j                            
                        elif i != set1[0][0] and j != set1[0][1]:
                            set1[1][0] = i
                            set1[1][1] = j
                            set2[1][0] = i
                            set2[0][1] = j
        #Выбрали индексы нужные и теперь определяем оптимальный набор
        val1 = matrix[set1[0][0]][set1[0][1]]*matrix[set1[1][0]][set1[1][1]]
        val2 = matrix[set2[0][0]][set2[0][1]]*matrix[set2[1][0]][set2[1][1]]
        if val1>val2: 
            result = set1
        else: result = set2
        
        #Теперь согласно оптимальному набору изменяем порядок
        for i in range(3):
            order[result[0][0]] = result[0][1]
            order[result[1][0]] = result[1][1]
        #Возвращаем порядок
    return order
    # elif n == 3:
    # #пока не придумала решения для такого типа ситуаций
    # #Да и не могу пока представить
    # #пытаюсь построить матрицу и не получается чтобы не было ни одного максимума и по строке и по столбцу
    #     order = [0,1,2]

In [107]:
#Провеееркааааа

# with open("D:/test_changed.pickle","rb") as f:
#     tmp=pickle.load(f)
#inputText = ["Петрович Птров Ивн"]

inputText = ["Петров Кирилл Анатольевич",
             "Александр Иванович Сидоров",
              "Бугазявкин Петре Петрович",
              "Синюевич Анатольевич Иван",
              "Иванович Иванов Иван",
              "Сидорок Анатоилй Петрович",
              "Кириллович Иван Соколов"]

testing_res=NotBruteAtAll(inputText)
for fio in testing_res:
    print(fio)

Петров Кирилл Анатольевич
Сидоров Александр Иванович
Бугазявкин Петр Петрович
Зинкевич Иван Анатольевич
Иванов Иван Иванович
Сидоров Анатолий Петрович
Соколов Иван Кириллович


In [39]:
def GetOrder(matrix): 
    #Определяет порядок элементов по матрице, в которой по одному элементу на строку и столбец
    order = [0,0,0]
    for j in range(3):
        for i in range(3):
            if (matrix[i][j]!=0):
                order[j] = i
                break
    return order

In [40]:
def СheckMatrix(matrix): 
    #Если у нас в одной строке и столбце только по одному элементу, то возвращает True
    for i in range(3):
        k2 = 0
        for j in range(3):
            if (matrix[i][j] != 0): k2+=1
        if (k2 != 1):
            return False
    return True

In [41]:
def СheckStrictMatrix(matrix):
    newMatrix = copy.deepcopy(matrix)
    matrix = copy.deepcopy(matrix)
    #Сначала приводит матрицу к значениям только из 0 и 1, затем выполняет рекурсивное изменение
    for i in range(3):
        for j in range(3):
            if(matrix[i][j]>0): matrix[i][j]=1
    matrix = recursiveChange(matrix, None)
    for i in range(3):
        for j in range(3):
            newMatrix[i][j] = newMatrix[i][j]*matrix[i][j]
    return newMatrix

In [42]:
def recursiveChange(matrix, matrixOld): 
    #Рекурсивное изменение значений в матрице - преобразует матрицу 
    # [[1,0,1],                [[0,0,1],
    #  [1,1,1],     в матрицу   [0,1,0],
    #  [1,0,0]]                 [1,0,0]]
        
    if(matrix==matrixOld): return matrix
    matrixOld = matrix
    counters = [0,0,0]
    for i in range(3):
        counters[i] = matrix[i].count(1)
        
    if counters.count(1)==3: return matrix
    if counters.count(1)==2 and counters.count(0)==1:
        counters2 = [False,False,False]
        for j in range(3):
            flag = True
            for i in range(3):
                if (matrix[i][j] != 0):
                    flag = False
                    break
            counters2[j] = flag
        i = counters.index(0)
        j = counters2.index(True)
        matrix[i][j] = 1
        return recursiveChange(matrix, matrixOld)
    
    if counters.count(1)==1:
        k = counters.index(1)
        j = matrix[k].index(1)
        for i in range(3):
            if (i != k):
                matrix[i][j] = 0
        return recursiveChange(matrix, matrixOld)
    if (counters.count(1)==2 and matrix[counters.index(1)].index(1)!=
            matrix[counters.index(1, counters.index(1)+1)].index(1)):
        k = counters.index(1, counters.index(1)+1)
        j = matrix[k].index(1)
        for i in range(3):
            if (i != k):
                matrix[i][j] = 0
        
        k = counters.index(1)
        j = matrix[k].index(1)
        for i in range(3):
            if (i != k):
                matrix[i][j] = 0
                
        return recursiveChange(matrix, matrixOld)
    return matrix

In [43]:
def GetMaxWordsValues(matrixValues, matrixWords):
    #На входе получает матрицу вероятностей, и матрицу слов, соответсвующих вероятностям
    #На выходе выдает массивы порядок, список слов и их вероятностей. Отбирает их по максимальному значению
    order = [None,None,None]
    words = [None,None,None]
    values = [0,0,0]
    for j in range(3):
        maxI = 0
        for i in range(3):
            if (matrixValues[i][j]>matrixValues[maxI][j]):
                maxI = i
        if (maxI not in order):
            order[j] = maxI
            words[j] = matrixWords[maxI][j]
            values[j] = matrixValues[maxI][j]
        #else: return False #исправить
        else:
            order[j] = 0
            words[j] = ""
            values[j] = 0
    return order, words, values

In [44]:
def StrictCheck(words): #строгая проверка
    #          F I O
    result = [[0,0,0],
              [0,0,0],
              [0,0,0]]
    for i in range(3):
        w = words[i].strip()
        for j in range(3):
            if w in bases[j]:
                result[i][j] = bases[j][w]
        
        # if (w in all_surnames): 
        #     result[i][0] = all_surnames[w]    
        # if (w in all_names):
        #     result[i][1] = all_names[w]        
        # if (w in all_patronymics):
        #     result[i][2] = all_patronymics[w]
    return result

In [45]:
def forReplaceCheck(w, list):
    #Отдельный метод, составляет списки слов с расстоянием 1 и 2
    #После этого сравнивает максимальные значения списокв, и возвращает слово с максимальной вероятностью 
    res = [{"": 0},{"": 0}]
    for l in list:
        dist = False
        if not (len(l) > len(w)+2 and len(l) < len(w)+2) :
            #dist = levenshtein(word, dictWord)
            dist = damerau(w, l)            
            if not (dist == 1 or dist==2):
                dist = False
        if (dist != False):
            res[int(dist)-1][l] = list[l]*10**(-1*dist)

    key1, value1 = max(res[0].items(), key=lambda x:x[1])
    key2, value2 = max(res[1].items(), key=lambda x:x[1])
    
    if (value1>value2):
        return key1, value1
    else: return key2, value2

In [46]:
def levenshtein(s, t): #расстояние Левенштейна
    if s == t: return 0
    elif len(s) == 0: return len(t)
    elif len(t) == 0: return len(s)
    v0 = [None] * (len(t) + 1)
    v1 = [None] * (len(t) + 1)   
    for i in range(len(v0)):
        v0[i] = i
    for i in range(len(s)):
        v1[0] = i + 1
        for j in range(len(t)):
            cost = 0 if s[i] == t[j] else 1
            v1[j + 1] = min(v1[j] + 1, v0[j + 1] + 1, v0[j] + cost)
        for j in range(len(v0)):
            v0[j] = v1[j]
    return v1[len(t)]

In [47]:
def damerau(s, t): #расстояние Дамерау-Левенштейна (с перестановкой)
    if s == t: return 0
    elif len(s) == 0: return len(t)
    elif len(t) == 0: return len(s)
    
    deleteCost = 1
    insertCost = 1
    replaceCost = 1
    transposeCost = 1
    s = " "+s
    t = " "+t
    M = len(s)
    N = len(t)
    d = [list(range(N))]
    for i in range(1,M):
        d.append([])
        for j in range(N):
            d[i].append(0)
        d[i][0] = i
        
    for i in range(1,M):
        for j  in range(1,N):          
            # Стоимость замены
            if (s[i] == t[j]):
                d[i][j] = d[i-1][j-1]
            else:
                d[i][j] = d[i-1][j-1] + replaceCost                   
            d[i][j] = min(
                             d[i][j],                               # замена
                             d[i-1][j] + deleteCost,                # удаление
                             d[i][j-1] + insertCost                 # вставка               
                         )
            if(i > 1 and j > 1 and s[i] == t[j-1] and s[i-1] == t[j]):
                d[i][j] = min(
                                  d[i][j],
                                  d[i-2][j-2] + transposeCost         # транспозиция
                             )
    return d[M-1][N-1]

In [48]:
def GrammaCheck(words): 
    #Проверка по грамматике - проверяются окончания отчества и фамилии
    #          F I O
    result = [[0,0,0],
              [0,0,0],
              [0,0,0]]

    for i in range(3):
        w = words[i].strip()
        result[i][0] = checkSurnames(w)
        result[i][2] = checkPatronymic(w)

    k = 0
    for i in range(3):
        flag = True
        for j in range(3):
            if (result[i][j] != 0): 
                flag = False 
                break
        if(flag):
            result[i][1] = 1
            k+=1
    if(k==0):
        for i in range(3): result[i][1] = 0.33
    elif(k>1): 
        for i in range(3): result[i][1] = result[i][1]/k
        
    #Теперь все вероятности домножаем на некий коэффициент - коэффициент ценности грамматической проверки
    #У меня это 0.01, пока что
    
    for i in range(3):
        for j in range(3):
            result[i][j] *= 0.01
    return result

In [49]:
def checkSurnames(s):
    #Проверка окончаний фамилий
    pattern = '\w*(ов|ова|ев|ёв|ева|ёва|ив|ин|ина|ын|их|ых|ский|цкий|ая|ко|дзе|онок|ян)$'
    if (re.match(pattern,s)): return 1
    else: return 0

In [50]:
def checkPatronymic(s):
    #Проверка окончаний отчеств
    pattern = '\w*(ович|евич|ич|овна|евна|ична|инична)$'
    if (re.match(pattern,s)):
        return 1
    else: return 0

In [51]:
#               F     I    O
statistics = [[0.50,0.45,0.05],
              [0.10,0.50,0.40],
              [0.40,0.05,0.55]]
# FIO: 0.50, IOF: 0.40, IFO: 0.15, ElseOptions: 0.05
#пока что совсем от балды, простите
def SetStatistics(order):
    for i in range(3):
        for j in range(3):
            if (j == order[i]): 
                statistics[i][j] += 0.002
            else: statistics[i][j] -= 0.001
#Обновляет статистику с учетом результата выполнения алгоритма            

In [55]:
surnames = {}
names = {}
patronymics = {}

path_to_surnames = "D:/surnames_extended_rate.pickle"
path_to_names = "D:/names_extended_rate.pickle"
path_to_patronymics = "D:/patronymics_extended_rate.pickle"
with open(path_to_surnames,"rb") as f:
    surnames=pickle.load(f)
with open(path_to_names,"rb") as f:
    names=pickle.load(f)
with open(path_to_patronymics,"rb") as f:
    patronymics=pickle.load(f)  
    
#Временные меры, пока мы не приведем базы к конечной форме, чтобы не изменять код каждый раз в зависимости от изменения структуры списков
all_surnames = {}
all_names = {}
all_patronymics = {}
for s in surnames:
    all_surnames[s] = surnames[s][1]
for s in names:
    all_names[s] = names[s][2]
for s in patronymics:
    all_patronymics[s] = patronymics[s][2]
    
bases = [all_surnames, all_names, all_patronymics]   

In [55]:
def NotBruteEnough(tmp): #Старый метод
    res = []

    for t in tmp:
        name = ""
        surname = ""
        patronymic = ""
        t = t.lower()
        words = t.split(" ")

        for w in words:
            w.strip()
            if (name == ""):
                if (w in all_names):
                    name = w

            if (patronymic == ""):
                if (w in all_patronymics):
                    patronymic = w

            if (name != w and patronymic != w and checkSurnames(w) and surname == ""):
                surname = w

        if (name in words): words.remove(name)
        if (patronymic in words): words.remove(patronymic)
        if (surname in words): words.remove(surname)

        if (name == "") & (len(words) > 0):
            foundWord = ""
            for w in words:
                minDist = len(w)
                for n in all_names:
                    dist = levenshtein(w, n)
                    if dist < minDist:
                        minDist = dist
                        name = n.title() #+ " (" + str(round((1 - minDist / len(w)) * 100)) + "% " + w + ")"
                        foundWord = w
            words.remove(foundWord)

        if (patronymic == "") & (len(words) > 0):
            foundWord = ""
            for w in words:
                minDist = len(w)
                for p in all_patronymics:
                    dist = levenshtein(w, p)
                    if dist < minDist:
                        minDist = dist
                        patronymic = p.title() #+ " (" + str(round((1 - minDist / len(w)) * 100)) + "% " + w + ")"
                        foundWord = w
            words.remove(foundWord)

        if (surname == "")& (len(words) > 0):
            surname = words[0]

        name = name.title()
        surname = surname.title()
        patronymic = patronymic.title()
        res.append(surname + " " + name + " " + patronymic)

    return res

In [56]:
#модуль с полами Пока не используем
genderTuple = ('male', 'female', 'unknown') #Неизменяемый массив значений пола

def checkSurnamesGender(s):
    #Проверяет пол исходя из окончания фамилии
    patternMale = '\w*(ов|ев|ий|ын|ин)$'
    patternFem ='\w*(ова|ева|ая|ина|ына)$'
    patternUnknown = '\w*(их|ых|ко|ук|юк|ун|ний|ный|чай|ий|а|ич|ов|ук|ик|ски|ка|ски|цки|дзки)$'
    
    if (re.match(patternMale,s)):
        return genderTuple[0]
    elif (re.match(patternFem,s)):
        return genderTuple[1]
    elif (re.match(patternUnknown,s)):
        return genderTuple[2]
    else: return genderTuple[2] #Возможно сделать другой вывод

def changeGenderSurname(s):
    #Меняет пол фамилии
    patternMaleA = '\w*(ов|ев|ин|ын)$'
    patternFemA ='\w*(ова|ева|ина|ына)$'
    patternMaleB = '\w*(ий)$'
    patternFemB = '\w*(ая)$'
    
    if(re.match(patternMaleA, s)):
        return s + 'a'
    elif (re.match(patternMaleB, s)):       
        s += ' '
        return s.replace('ий ', 'ая')
    elif (re.match(patternFemA, s)):
        return s[0: len(s)-1]
    elif (re.match(patternFemB, s)): 
        s += ' '
        return s.replace('ая ', 'ий')
    else: return s

def checkPatronymicGender(s):
    #Проверяет пол исходя из окончания отчества
    patternMale = '\w*(ович|евич|ич)$'
    patternFem ='\w*(овна|евна|ична|инична)$'
   
    if (re.match(patternMale,s)):
        return genderTuple[0]
    elif (re.match(patternFem,s)):
        return genderTuple[1]
    else: return 'No'    

def changeGenderPatronymic(s):
    #Меняет пол отчества
    patternMaleA = '\w*(ович|евич)$'
    patternFemA ='\w*(овна|евна)$'
    patternMaleB = '\w*(ич)$'
    patternFemB = '\w*(ична|инична)$'

    if(re.match(patternMaleA, s)):
        s += ' '
        return s.replace('ич ', 'на')
    if(re.match(patternFemA, s)):
        s += ' '
        return s.replace('на ', 'ич')     
    if(re.match(patternMaleB, s)):
        return s + 'на'   
    if(re.match(patternFemB, s)):
        s += ' '
        s = s.replace('инична ', 'ич')
        s = s.replace('ична ', 'ич')
        return s    
    else: return s